In [ ]:
## Load libraries
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
np.set_printoptions(precision = 2)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
# Generate artificial data with 5 samples, 4 features per sample
# and 3 output classes
num_samples = 5 # number of samples
num_features = 4 # number of features (a.k.a. dimensionality)
num_labels = 3 # number of output labels
# Data matrix (each column = single sample)
X = np.random.choice(np.arange(3, 10), size = (num_features, num_samples), replace = True)
# Class labels
y = np.random.choice([0, 1, 2], size = num_samples, replace = True)
print(X)
print('------')
print(y)
print('------')
# One-hot encode class labels
y = tf.keras.utils.to_categorical(y)
print(y)

[[7 6 3 5 8]
 [5 8 6 7 8]
 [6 5 3 7 7]
 [3 4 7 4 5]]
------
[0 2 1 2 0]
------
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


---

A generic layer class with forward and backward methods

----

In [ ]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None

  def forward(self, input):
    pass

  def backward(self, output_gradient, learning_rate):
    pass

---

The softmax classifier steps for a generic sample $\mathbf{x}$ with (one-hot encoded) true label $\mathbf{y}$ (3 possible categories) using a randomly initialized weights matrix (with bias abosrbed as its last last column):

1. Calculate raw scores vector for a generic sample $\mathbf{x}$  (bias feature added): $$\mathbf{z} = \mathbf{Wx}.$$
2. Calculate softmax probabilities (that is, softmax-activate the raw scores) $$\mathbf{a} = \text{softmax}(\mathbf{z})\Rightarrow\begin{bmatrix}a_0\\a_1\\a_2\end{bmatrix}= \text{softmax}\left(\begin{bmatrix}z_0\\z_1\\z_2\end{bmatrix}\right)=\begin{bmatrix}\frac{e^{z_0}}{e^{z_0}+e^{z_1}+e^{z_2}}\\\frac{e^{z_1}}{e^{z_0}+e^{z_1}+e^{z_2}}\\\frac{e^{z_2}}{e^{z_0}+e^{z_1}+e^{z_2}}\end{bmatrix}$$
3. Softmax loss for this sample is (where output label $y$ is not yet one-hot encoded)
$$\begin{align*}L &=  -\log([a]_y) \\&= -\log\left(\left[\text{softmax}(\mathbf{z})\right]_y\right)\\ &= -\log\left(\left[\text{softmax}(\mathbf{Wx})\right]_y\right).\end{align*}$$
4. Predicted probability vector that the sample belongs to each one of the output categories is given a new name $$\hat{\mathbf{y}} = \mathbf{a}.$$
5. One-hot encoding the output label $$\underbrace{y\rightarrow\mathbf{y}}_{\text{e.g.}\,2\,\rightarrow\begin{bmatrix}0\\0\\1\end{bmatrix}}$$ results in the following representation for the softmax loss for the sample which is also referred to as the categorical crossentropy (CCE) loss:
$$\begin{align*}L &= L\left(\mathbf{y},\hat{\mathbf{y}}\right)=\sum_{k=0}^2-y_k\log\left(\hat{y}_k\right)\end{align*}.$$
5. Calculate the gradient of the loss for the sample w.r.t. weights by following the computation graph from top to bottom (that is, backward):
$$\begin{align*} L\\{\color{yellow}\downarrow}\\ \hat{\mathbf{y}} &= \mathbf{a}\\{\color{yellow}\downarrow}\\\mathbf{z}\\{\color{yellow}\downarrow}\\\mathbf{W}\end{align*}$$
$$\begin{align*}\Rightarrow \nabla_\mathbf{W}(L) &= \nabla_\mathbf{W}(\mathbf{z}) \times\nabla_\mathbf{z}(\mathbf{a})\times\nabla_\mathbf{a}(L)\\&= \underbrace{\nabla_\mathbf{W}(\mathbf{z})}_\text{first term} \times\underbrace{\nabla_\mathbf{z}(\mathbf{a})}_\text{second to last term}\times\underbrace{\nabla_\hat{\mathbf{y}}(L)}_\text{last term}.\end{align*}$$
7. Now focus on the last term $\nabla_\hat{\mathbf{y}}(L)$:
$$\begin{align*}\nabla_\hat{\mathbf{y}}(L) &=\begin{bmatrix}\nabla_{\hat{y}_0}(L)\\\nabla_{\hat{y}_1}(L)\\\nabla_{\hat{y}_2}(L)\end{bmatrix} = \begin{bmatrix}-y_0/\hat{y}_0\\-y_1/\hat{y}_1\\-y_2/\hat{y}_2.\end{bmatrix}\end{align*}$$
8. Now focus on the second to last term $\nabla_\mathbf{z}(\mathbf{a})$:
$$\begin{align*}\nabla_\mathbf{z}(\mathbf{a}) &= \nabla_\mathbf{z}\left(\begin{bmatrix}a_0\\a_1\\a_2\end{bmatrix}\right)\\ &= \begin{bmatrix}\nabla_\mathbf{z}(a_0)&\nabla_\mathbf{z}(a_1)&\nabla_\mathbf{z}(a_2)\end{bmatrix} \\&= \begin{bmatrix}\nabla_{z_0}(a_0)&\nabla_{z_0}(a_1)&\nabla_{z_0}(a_2)\\\nabla_{z_1}(a_0)&\nabla_{z_1}(a_1)&\nabla_{z_1}(a_2)\\\nabla_{z_2}(a_0)&\nabla_{z_2}(a_1)&\nabla_{z_2}(a_2)\end{bmatrix}\\&=\begin{bmatrix}a_0(1-a_0)&-a_1a_0&-a_2a_0\\-a_0a_1&a_1(1-a_1)&-a_2a_1\\-a_0a_2&-a_1a_2&a_2(1-a_2)\end{bmatrix}.\end{align*}$$
9. On Monday, we will focus on the first term to complete the gradient calculation using the computation graph.


---

In [ ]:
## Softmax activation class
class Softmax(Layer):
  def forward(self, input):
    self.output = np.array(tf.nn.softmax(input))

  def backward(self, output_gradient, learning_rate = None):
    return(np.dot((np.identity(np.size(self.output))-self.output.T) * self.output, output_gradient))

In [ ]:
## Define the loss function and its gradient
def cce(y, yhat):
  return(-np.sum(y*np.log(yhat)))

def cce_gradient(y, yhat):
  return(-y/yhat)

# TensorFlow in-built function for categorical crossentropy loss
#cce = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
# Step-1: add the bias feature to all the samples
X = np.vstack([X, np.ones((1, num_samples))])

In [ ]:
## Train the 0-layer neural network using batch training with
## batch size = 1

# Steps: run over each sample, calculate loss, gradient of loss,
# and update weights.


# Step-2: initialize the entries of the weights matrix randomly
W = np.random.normal(0, 1, (num_labels, num_features))
W = np.hstack([W, 0.01*np.ones((num_labels, 1))])

# Step-3: create softmax layer object softmax
softmax = Softmax()

# Step-4: run over each sample
for i in range(X.shape[1]):
  # Step-5: forward step
  # (a) Raw scores z = Wx = np.dot(W, X[:, i])
  z = np.dot(W, X[:, i])
  # (b) Softmax activation: softmax.forward(z)
  softmax.forward(z)
  # (c) Calculate cce loss for sample: cce(y[i, :], softmax.output)
  loss = cce(y[i, :], softmax.output)
  # (d) Print cce loss
  print(loss)

  # Step-6: backward step
  # (a) Calculate the gradient of the sample loss w.r.t. input of the
  # softmax layer:
  grad = cce_gradient(y[i, :], softmax.output)
  grad = softmax.backward(output_gradient = grad)
  grad = grad.reshape(-1, 1) * X[:, i].reshape(-1, 1).T
  # (d) Print gradient
  print(grad)
  # Gradient descent step
  learning_rate = 1e-03
  W = W + learning_rate * (-grad)

17.090754891083982
[[-7.00e+00 -5.00e+00 -6.00e+00 -3.00e+00 -1.00e+00]
 [ 2.65e-07  1.89e-07  2.27e-07  1.13e-07  3.78e-08]
 [ 2.65e-07  1.89e-07  2.27e-07  1.13e-07  3.78e-08]]
3.071028915466482e-05
[[ 6.00e+00  8.00e+00  5.00e+00  4.00e+00  1.00e+00]
 [ 6.00e+00  8.00e+00  5.00e+00  4.00e+00  1.00e+00]
 [-1.84e-04 -2.46e-04 -1.54e-04 -1.23e-04 -3.07e-05]]
10.643741633265126
[[ 7.15e-05  1.43e-04  7.15e-05  1.67e-04  2.38e-05]
 [-3.00e+00 -6.00e+00 -3.00e+00 -7.00e+00 -1.00e+00]
 [ 7.15e-05  1.43e-04  7.15e-05  1.67e-04  2.38e-05]]
0.016147963313500976
[[ 4.92  6.89  6.89  3.94  0.98]
 [ 4.92  6.89  6.89  3.94  0.98]
 [-0.08 -0.11 -0.11 -0.06 -0.02]]
27.235044150839975
[[-8.00e+00 -8.00e+00 -7.00e+00 -5.00e+00 -1.00e+00]
 [ 1.19e-11  1.19e-11  1.04e-11  7.43e-12  1.49e-12]
 [ 1.19e-11  1.19e-11  1.04e-11  7.43e-12  1.49e-12]]
